In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, length, from_json, expr, split, lit, to_date, explode, count, lower, trim, regexp_replace, substring
from pyspark.sql.types import StringType, StructType, StructField, MapType, ArrayType, DoubleType, DateType, IntegerType

In [2]:
import xml.etree.ElementTree as ET
import requests
import os
import collections
import time
import html

import pandas as pd
import yake
import numpy as np
from tqdm import tqdm

In [3]:
import utils

In [4]:
AWS_ACCESS_KEY_ID = 'test_key_id'
AWS_SECRET_ACCESS_KEY = 'test_access_key'
HOST = 's3'
ENDPOINT_URL = f'http://{HOST}:4566'

TEMP_DIR = './local_data'

In [5]:
spark = SparkSession.builder.getOrCreate()

# Collect data from S3

In [6]:
CONTENTS = ['movie/review', 'movie/info', 'boardgame/boardgame', 'boardgame/collection', 'videogame']#, 'anime']
# CONTENTS = ['movie/review']#, 'boardgame', 'videogame']#, 'anime']

In [7]:
def download_raw_data_of_content(content):
    print(f'Downloading raw-data of {content}...')
    s3 = utils.S3_conn()
    keys = s3.get_keys_with_prefix('raw-data', content)
    target_dir = f"{TEMP_DIR}/{content}"
    
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
    
    for key in tqdm(keys):
        local_file_path = f'{target_dir}/{key[len(content) + 1:]}'# os.path.join(target_dir, key[len(kind):])
        local_file_dir = os.path.dirname(local_file_path)
        if not os.path.exists(local_file_dir):
            os.makedirs(local_file_dir)
    
        s3.s3_client.download_file('raw-data', key, local_file_path)

In [66]:
%%time
for content in CONTENTS:
    download_raw_data_of_content(content)

100%|██████████| 1000/1000 [00:14<00:00, 70.83it/s]


100%|██████████| 1000/1000 [00:16<00:00, 61.20it/s]


100%|██████████| 1000/1000 [00:14<00:00, 67.29it/s]


100%|██████████| 657/657 [00:10<00:00, 61.73it/s] 


100%|██████████| 3/3 [00:02<00:00,  1.48it/s]

CPU times: user 29.5 s, sys: 6.16 s, total: 35.7 s
Wall time: 58.4 s


# Transform data

In [8]:
s3 = utils.S3_conn()

In [9]:
def store_processed_parquet(local_directory, prefix):
    bucket_name = 'processed-data'
    
    for root, dirs, files in tqdm(os.walk(local_directory)):
        for filename in files:
            # Construct the full local path
            local_path = os.path.join(root, filename)
            
            # Construct the relative path for S3
            relative_path = os.path.relpath(local_path, local_directory)
            s3_path = os.path.join(prefix, relative_path).replace("\\", "/")  # Ensure Unix-style paths for S3
            
            # Upload the file to S3
            s3.s3_client.upload_file(local_path, bucket_name, s3_path)
            # print(f'Uploaded {local_path} to s3://{bucket_name}/{s3_path}\n')

## Boardgames

In [10]:
BOARDGAME_USERS_XML_PATH = './local_data/boardgame/collection'
BOARDGAME_USERS_PARQUET_PATH = './local_data/boardgame/processed_data/boardgame_users.parquet'
BOARDGAME_CONTENT_XML_PATH = './local_data/boardgame/boardgame'
BOARDGAME_CONTENT_PARQUET_PATH = './local_data/boardgame/processed_data/boardgame_content.parquet'

In [11]:
def xml_boardgame_to_dataframe():
    df_content_id = []
    df_content_description = []
    df_content_year = []

    for folder in os.listdir(BOARDGAME_CONTENT_XML_PATH):
        with open(f"{BOARDGAME_CONTENT_XML_PATH}/{folder}/1.xml", 'r') as f:
            r_text = f.read()
        df_content_id.append(folder)
        root = ET.fromstring(r_text)
        for bg in root:
            for field in bg:
                if field.tag == 'description':
                    df_content_description.append(html.unescape(field.text))
                if field.tag == 'yearpublished':
                    df_content_year.append(int(field.attrib['value']))

    return pd.DataFrame({
        'content_id': pd.Series(df_content_id, dtype='str'),
        'description': pd.Series(df_content_description, dtype='str'),
        'release_year': pd.Series(df_content_year, dtype='Int16')
    })

In [12]:
def create_boardgame_content_parquet():
    schema = StructType([
        StructField("content_id", StringType(), True),
        StructField("description", StringType(), True),
        StructField("release_year", IntegerType(), True)
    ])
    
    df = xml_boardgame_to_dataframe()
    df['description'] = df['description'].astype('str')
    df = df.replace([np.nan], [None])
    
    boardgame_content = (
        spark
        .createDataFrame(df, schema=schema)
        .withColumn('type', lit('boardgame'))
    )
    
    # Save parquet to processed-data zone
    boardgame_content.write.mode('overwrite').parquet(BOARDGAME_CONTENT_PARQUET_PATH)

In [13]:
def get_boardgame_content_df():
    boardgame_content = spark.read.parquet(BOARDGAME_CONTENT_PARQUET_PATH)
    return boardgame_content

In [14]:
def xml_collection_to_dataframe(xml_file) -> pd.DataFrame:
    with open(xml_file, 'r') as f:
        r_text = f.read()
        root = ET.fromstring(r_text)

    df_user_id = []
    df_type = []
    df_content_id = []
    df_rating = []
    df_rating_date = []
    
    for bg in root:
        bg_name = bg[0].text
        coll_id = bg.attrib['collid']  # I don't really know what this is, but I guess it is the id of this instance of the boardgame in the list
        object_id = bg.attrib['objectid']  # This is the boardgame identifier

        rating_val = None
        for field in bg:
            if field.tag == 'stats':
                rating_val = field[0].attrib['value']
                if rating_val == 'N/A':
                    rating_val = None
            if field.tag == 'yearpublished':
                year_published = field.text
            if field.tag == 'status':
                date_of_rating = field.attrib['lastmodified']  # Not really the rating date, but it is as close as possible with the current information.

        # print(user_id, 'boardgame', object_id, rating_val, date_of_rating)
        # print(bg_name, rating_val, year_published, coll_id, object_id)
        df_user_id.append(xml_file.split('/')[-1][:-4])
        df_type.append('boardgame')
        df_content_id.append(object_id)
        df_rating.append(rating_val)
        df_rating_date.append(date_of_rating)

    return pd.DataFrame({
        'user_id': pd.Series(df_user_id, dtype='str'),
        'type': pd.Series(df_type, dtype='category'),
        'content_id': pd.Series(df_content_id, dtype='str'),
        'rating': pd.Series(df_rating, dtype='float64'),
        'rating_date': pd.Series(df_rating_date, dtype='datetime64[ms]')
    })

In [15]:
def create_boardgame_users_parquet():
    if not os.path.exists(BOARDGAME_USERS_PARQUET_PATH):
        os.makedirs(BOARDGAME_USERS_PARQUET_PATH)
        
    for xml in filter(lambda x: x.endswith('.xml'), os.listdir(BOARDGAME_USERS_XML_PATH)):
        try:
            df = xml_collection_to_dataframe(f'{BOARDGAME_USERS_XML_PATH}/{xml}')
            parquet_path = f'{BOARDGAME_USERS_PARQUET_PATH}/{xml[:-4]}.parquet'
            df.to_parquet(parquet_path)
        except Exception as e:
            print(e)
            print(f'Error: Invalid xml file: {xml}')

In [16]:
def get_boardgame_users_df():
    boardgame_users = spark.read.parquet(BOARDGAME_USERS_PARQUET_PATH)
    return boardgame_users

In [17]:
create_boardgame_users_parquet()

'collid'
Error: Invalid xml file: Century.xml
'collid'
Error: Invalid xml file: Icythistle.xml
'collid'
Error: Invalid xml file: ItsCharlieVP.xml
'collid'
Error: Invalid xml file: nugenet.xml
'collid'
Error: Invalid xml file: marioymia.xml
'collid'
Error: Invalid xml file: RobMcWiz.xml
'collid'
Error: Invalid xml file: zigooloo.xml
'collid'
Error: Invalid xml file: Halenor.xml


In [18]:
boardgame_users = get_boardgame_users_df()
boardgame_users.show(5)
boardgame_users.printSchema()

+-----------+---------+----------+------+-------------------+
|    user_id|     type|content_id|rating|        rating_date|
+-----------+---------+----------+------+-------------------+
|zefquaavius|boardgame|    322232|   6.0|2023-08-01 14:52:32|
|zefquaavius|boardgame|    296402|   8.0|2023-08-02 14:18:24|
|zefquaavius|boardgame|    336537|  null|2023-08-02 14:18:38|
|zefquaavius|boardgame|    314445|  null|2023-08-02 14:18:54|
|zefquaavius|boardgame|    296404|  null|2023-08-02 14:19:11|
+-----------+---------+----------+------+-------------------+
only showing top 5 rows

root
 |-- user_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- content_id: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- rating_date: timestamp_ntz (nullable = true)



In [19]:
create_boardgame_content_parquet()

In [20]:
boardgame_content = get_boardgame_content_df()
boardgame_content.show(5)
boardgame_content.printSchema()

+----------+--------------------+------------+---------+
|content_id|         description|release_year|     type|
+----------+--------------------+------------+---------+
|      1117|Classic bidding a...|        1985|boardgame|
|      1086|In Treasure of th...|        1998|boardgame|
|    107704|Fief is a game of...|        2011|boardgame|
|     10226|Thunder at the Cr...|        1993|boardgame|
|    104351|Raphia\n\nThe Bat...|        2011|boardgame|
+----------+--------------------+------------+---------+
only showing top 5 rows

root
 |-- content_id: string (nullable = true)
 |-- description: string (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- type: string (nullable = true)



In [21]:
store_processed_parquet(BOARDGAME_USERS_PARQUET_PATH, prefix='boardgame')

1it [00:06,  6.28s/it]


In [22]:
store_processed_parquet(BOARDGAME_CONTENT_PARQUET_PATH, prefix='boardgame')

1it [00:00,  5.21it/s]


# Movie

In [25]:
MOVIE_BASE_PARQUET_PATH = './local_data/movie/review'
MOVIE_BASE_INFO_PATH = './local_data/movie/info'
MOVIE_USERS_PARQUET_PATH = "./local_data/movie/processed_data/movie_users.parquet"
MOVIE_CONTENT_PARQUET_PATH = "./local_data/movie/processed_data/movie_content.parquet"

In [44]:
def create_movie_users_parquet():
    schema = ArrayType(
        StructType([
            StructField("author", StringType(), True),
            StructField("author_details", StructType([
                StructField("rating", StringType(), True)
            ]), True),
            StructField("created_at", StringType(), True),
        ])
    )
    
    movie_users = spark.read.parquet(MOVIE_BASE_PARQUET_PATH)\
              .filter(length("results")>2)\
              .withColumn("results_test", col('results'))\
              .withColumn("results_parsed", from_json(col("results_test"), schema))\
              .withColumn("result_exploded", explode(col("results_parsed")))\
              .withColumn('result_exploded', col("result_exploded").cast(StringType()))
    
    split_col = split(movie_users['result_exploded'], ', ')
    
    movie_users = movie_users.withColumn('author', split_col.getItem(0)) \
               .withColumn('author', expr("substring(author,2, length(author) -1)")) \
               .withColumn('rating', split_col.getItem(1)) \
               .withColumn("rating", expr("substring(rating, 2, length(rating) - 2)"))\
               .withColumn("rating", col('rating').cast(DoubleType()))\
               .withColumn('rating_date', split_col.getItem(2))\
               .withColumn('rating_date', expr("substring(rating_date,1, length(rating_date) -1)"))\
               .withColumn("rating_date", to_date(col("rating_date"), "yyyy-MM-dd'T'HH:mm:ss.SSS'Z'"))\
               .select(col('author').alias('user_id'), lit('movie').alias('type'), col('id').alias('content_id').cast(StringType()), 'rating', 'rating_date')

    if not os.path.exists(MOVIE_USERS_PARQUET_PATH):
        os.makedirs(MOVIE_USERS_PARQUET_PATH)

    movie_users.repartition(1).write.mode('overwrite').parquet(MOVIE_USERS_PARQUET_PATH)

In [38]:
def create_movie_content_parquet():
    if not os.path.exists(MOVIE_CONTENT_PARQUET_PATH):
        os.makedirs(MOVIE_CONTENT_PARQUET_PATH)
    movie_content = (
        spark
        .read.parquet(MOVIE_BASE_INFO_PATH)
        .select(col('id').alias('content_id'), col('overview').alias('description'), col('release_date').alias('release_year'))
        .withColumn('release_year', substring("release_year", 1, 4))
        .repartition(1).write.mode('overwrite').parquet(MOVIE_CONTENT_PARQUET_PATH)
    )
    movie_content.repartition(1).write.mode('overwrite').parquet(MOVIE_CONTENT_PARQUET_PATH)

In [41]:
def get_movie_users_df():
    movie_users = spark.read.parquet(MOVIE_USERS_PARQUET_PATH)
    return movie_users

In [34]:
def get_movie_content_df():
    movie_content = spark.read.parquet(MOVIE_CONTENT_PARQUET_PATH)
    return movie_content

In [45]:
create_movie_users_parquet()

In [39]:
create_movie_content_parquet()

In [46]:
movie_users = get_movie_users_df()
movie_users.show(5)
movie_users.printSchema()

+------------------+-----+----------+------+-----------+
|           user_id| type|content_id|rating|rating_date|
+------------------+-----+----------+------+-----------+
|        John Chard|movie|       576|  10.0| 2017-02-10|
|      tmdb28039023|movie|       576|   6.0| 2022-08-28|
|Filipe Manuel Neto|movie|       576|   5.0| 2023-10-15|
|  Manuel São Bento|movie|    850165|   7.0| 2023-12-21|
|             r96sk|movie|    850165|   9.0| 2024-02-09|
+------------------+-----+----------+------+-----------+
only showing top 5 rows

root
 |-- user_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- content_id: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- rating_date: date (nullable = true)



In [47]:
movie_content = get_movie_content_df()
movie_content.show(5)
movie_content.printSchema()

+----------+--------------------+------------+
|content_id|         description|release_year|
+----------+--------------------+------------+
|     43969|Nogreh is a young...|        2003|
|    651102|Since its first p...|        1971|
|     80957|Brian, (Luke Goss...|        2011|
|    936897|Goldy is a spirit...|        2022|
|    146536|A journey back in...|        1986|
+----------+--------------------+------------+
only showing top 5 rows

root
 |-- content_id: long (nullable = true)
 |-- description: string (nullable = true)
 |-- release_year: string (nullable = true)



In [48]:
store_processed_parquet(MOVIE_USERS_PARQUET_PATH, prefix='movie')

1it [00:00, 15.24it/s]


In [49]:
store_processed_parquet(MOVIE_CONTENT_PARQUET_PATH, prefix='movie')

1it [00:00, 16.77it/s]


In [33]:
# GC had this code to enrich movie_users, I don't see the use yet so I am leaving this commented out.
# movie_users = movie_users.join(movie_content, ['content_id'],'left')
# movie_users.repartition(1).write.mode('overwrite').parquet(MOVIE_CONTENT_PARQUET_PATH)

In [50]:
# schema = ArrayType(
#     StructType([
#         StructField("author", StringType(), True),
#         StructField("author_details", StructType([
#             StructField("rating", StringType(), True)
#         ]), True),
#         StructField("created_at", StringType(), True),
#     ])
# )

# movie_users = spark.read.parquet(MOVIE_BASE_PARQUET_PATH)\
#           .filter(length("results")>2)\
#           .withColumn("results_test", col('results'))\
#           .withColumn("results_parsed", from_json(col("results_test"), schema))\
#           .withColumn("result_exploded", explode(col("results_parsed")))\
#           .withColumn('result_exploded', col("result_exploded").cast(StringType()))

# split_col = split(movie_users['result_exploded'], ', ')

# movie_users = movie_users.withColumn('author', split_col.getItem(0)) \
#            .withColumn('author', expr("substring(author,2, length(author) -1)")) \
#            .withColumn('rating', split_col.getItem(1)) \
#            .withColumn("rating", expr("substring(rating, 2, length(rating) - 2)"))\
#            .withColumn("rating", col('rating').cast(DoubleType()))\
#            .withColumn('rating_date', split_col.getItem(2))\
#            .withColumn('rating_date', expr("substring(rating_date,1, length(rating_date) -1)"))\
#            .withColumn("rating_date", to_date(col("rating_date"), "yyyy-MM-dd'T'HH:mm:ss.SSS'Z'"))\
#            .select(col('author').alias('user_id'), lit('movie').alias('type'), col('id').alias('content_id').cast(StringType()), 'rating', 'rating_date')

# movie_content = spark.read.parquet(MOVIE_BASE_INFO_PATH)\
#               .select(col('id').alias('content_id'),col('overview').alias('description'), col('release_date').alias('release_year'))\
#               .withColumn('release_year', substring("release_year", 1, 4))

# # movie_content.repartition(1).write.mode('overwrite').parquet("./parsed_data/movies_descript.parquet")

# # Re-lectura

# # dfIni = spark.read.parquet("./parsed_data/movies_user.parquet")
# # dfDesc = spark.read.parquet("./parsed_data/movies_descript.parquet")



# Anime

In [ ]:
animes = spark.read.parquet('./anime/data/anime.parquet').select('user_id','type','content_id','rating','rating_date')
animes.printSchema()

# Videogames

In [ ]:
#.schema('user_id STRING, type STRING, content_id string, rating DOUBLE, rating_date DATE')
videogames = (
    spark.read.parquet('./videogames/processed_data/steam_users.parquet')
    .withColumn('rating_date', to_date(col('rating_date'), 'yyyy-HH'))
    .select('user_id','type','content_id','rating','rating_date')
)
videogames.printSchema()

In [ ]:
videogames.show()

# Merging all content

In [50]:
merged_users = (
    boardgame_users
    .union(movie_users.select(['user_id', 'type', 'content_id', 'rating', 'rating_date']))
    # .union(animes)
    # .union(videogames)
    .withColumn('user_id', trim(lower(col('user_id'))))
    .withColumn('type', lower(col('type')))
)
merged_users.printSchema()
merged_users.sample(False, 0.1).orderBy(col('rating_date')).show()

root
 |-- user_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- content_id: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- rating_date: timestamp_ntz (nullable = true)

+------------+---------+----------+------+-------------------+
|     user_id|     type|content_id|rating|        rating_date|
+------------+---------+----------+------+-------------------+
|akapastorguy|boardgame|      1608|   6.0|2003-07-29 15:49:58|
|akapastorguy|boardgame|      3141|   7.0|2003-07-29 15:49:58|
|akapastorguy|boardgame|      1207|   5.0|2003-07-29 15:56:05|
|akapastorguy|boardgame|       295|   7.0|2003-07-30 21:11:56|
|akapastorguy|boardgame|      1545|   8.0|2003-07-30 21:19:27|
|akapastorguy|boardgame|       505|   7.0|2003-08-03 22:58:44|
|akapastorguy|boardgame|      1250|   4.0|2003-08-03 23:03:07|
|akapastorguy|boardgame|       102|   8.0|2003-08-03 23:11:22|
|akapastorguy|boardgame|      1117|   4.0|2003-08-03 23:11:22|
|akapastorguy|boardgame|      320

In [51]:
merged_users.groupBy(col('user_id')).count().sort(col('count'), ascending=False).show()

+------------------+-----+
|           user_id|count|
+------------------+-----+
|       zefquaavius| 5547|
|      akapastorguy| 4636|
|          naarnold| 3212|
|             tydel| 3171|
|         adrimetum| 2959|
|          doccabet| 2840|
|            huffa2| 2434|
|              muyf| 2372|
|           domi123| 2243|
|       donnie lama| 2224|
|kreikkaturkulainen| 2004|
|         doomfarer| 1994|
|            d0gb0t| 1974|
|    saxophonechapa| 1951|
|            landru| 1922|
|           okami31| 1913|
|            e1deno| 1903|
|          ryousuke| 1864|
|           bobondi| 1832|
|          riever32| 1831|
+------------------+-----+
only showing top 20 rows



In [52]:
(
    merged_users
    .select('user_id', 'type')
    .withColumn('user_id', trim(regexp_replace(lower(col('user_id')), '[^a-zA-Z0-9]', '')))
    .distinct()
    .groupBy(col('user_id'))
    .count()
    .sort(col('count'), ascending=False)
    .show()
)

+------------------+-----+
|           user_id|count|
+------------------+-----+
|      luckyoneputt|    1|
|         eagle1207|    1|
|            max667|    1|
|        frankbouch|    1|
|           nforget|    1|
|gemaskerdemuchacho|    1|
|         radziku87|    1|
|             megsg|    1|
|           juserre|    1|
|            wuchak|    1|
|         stmcguire|    1|
|          serranon|    1|
| angrytechnobadger|    1|
|           devrick|    1|
|          sjakkajs|    1|
|          garethmb|    1|
|         yurjinfox|    1|
|              p3te|    1|
|         gh0st2279|    1|
|        gulfcess23|    1|
+------------------+-----+
only showing top 20 rows



In [53]:
(
    merged_users
    .select('type')
    .groupBy(col('type'))
    .count()
    .sort(col('count'), ascending=False)
    .show()
)

+---------+------+
|     type| count|
+---------+------+
|boardgame|258496|
|    movie|    36|
+---------+------+



In [54]:
(
    merged_users
    .filter(merged_users['user_id'] == 'daimyo')
    .groupBy(col('type'))
    .count()
    .show()
)

+----+-----+
|type|count|
+----+-----+
+----+-----+



In [55]:
# Check what the RS does for the NULL values.
# - We could impute something, like the average score the user gives.

# Yake

In [124]:
# merged.show()

In [125]:
# boardgame_content.show()

In [126]:
# r = boardgame_content.rdd.map(lambda x: (x[2], get_kw(x[0])))

In [127]:
# spark.createDataFrame(r).show()

In [128]:
def get_kw(text):
   kw_extractor = yake.KeywordExtractor(
       lan='en',
       n=2,  # Max n-gram size
       top=5  # Number of keywords
   )
    
   return list(map(lambda x: str.lower(x[0]) if x else '', kw_extractor.extract_keywords(text)))


In [129]:
df = boardgame_content

In [130]:
rddK = df.rdd.map(lambda x: (x['content_id'], get_kw(x['description'])))
rddK = spark.createDataFrame(rddK).select(col('_1').alias('content_id'), col('_2').alias('keyword'))
dfK = (rddK.withColumn("keyword_1", expr("keyword[0]"))
                .withColumn("keyword_2", expr("keyword[1]"))
                .withColumn("keyword_3", expr("keyword[2]"))
                .withColumn("keyword_4", expr("keyword[3]"))
                .withColumn("keyword_5", expr("keyword[4]"))
                .select('content_id','keyword_1','keyword_2','keyword_3','keyword_4','keyword_5' )
      )
dfK.show()

+----------+--------------------+--------------------+----------------+-----------------+--------------------+
|content_id|           keyword_1|           keyword_2|       keyword_3|        keyword_4|           keyword_5|
+----------+--------------------+--------------------+----------------+-----------------+--------------------+
|      1117|       bluffing game|                 bid| classic bidding|           animal|             players|
|      1086|          move pawns|          outer path|    players move|      earthdevils|               earth|
|    107704|         middle ages|                fief|            ages|          players|              middle|
|     10226|     gettysburg july|                july|   george gordon|     gordon meade|               meade|
|    104351|         syrian wars|       antiochus iii|    modern rafah| seleucid kingdom|       battle fought|
|     10355|        golfing game|                game|       golf game|             golf|           judgement|
|

- [ ] connect directly spark to neo4j (using the right connector)
- [ ] maybe provide some analytics about the users' profile